In [1]:
import pandas as pd
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
%matplotlib inline
from textblob.classifiers import NaiveBayesClassifier
import re,string

In [2]:
NewTrain = pd.read_csv("C:\\Users\\navi\\Coba Coding 12 Juli 2020_menggunakan data beneran\\CrawlData_1_v4_Copy.csv", delimiter=',', encoding='utf-8')
NewTrain

,Tweet,Label
0,@FirstMediaCares ini kenapa internet rumah say...,pos
1,@IndiHome cara berhenti berlangganan bagaimana...,pos
2,@FirstMediaCares Telfon nmr di atas d suruh le...,neg
3,"@FirstMediaCares hi admin, sepertinya skrng su...",pos
4,ASSALAMUALAIKUM !!!! Nomer pelanggan 122114220...,pos
...,...,...
1167,bahasa ditulis penuh berhatihati tapi baca pen...,pos
1168,demmit sedih laaaaaa cerita menahan air mata m...,pos
1169,bahasa bintang jalan cerita bintang,pos
1170,tenggelam baitbait lemas dugaan luluh hatiku p...,net


In [3]:
NewTrain['Tweet'][27]

'RT @AdamMaulud: Provider internet paling jelek menurut kalian apa guys?  Kalo gw sih @IndiHome ,Kenapa? \r1.lemotnya gaada obat\r 2.keseringan…'

In [4]:
NewTrain = pd.DataFrame(NewTrain)
NewTrain["Tweet"] = NewTrain['Tweet'].astype(str) 

In [5]:
def preprocess(tweet):
    tweet = re.sub(r'RT','',tweet) #hapus tulisan "RT"
    tweet = re.sub(r'http\S+','',tweet)
    tweet = re.sub(r'@[\w]*','',tweet)#tweet = re.sub(r'@\S+','',tweet) #hapus username
    tweet = hapus_tanda(tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #hapus #tagger
    tweet = re.sub(r'\w*\d\w*', '',tweet).strip()#hapus angka dan angka yang berada dalam string
    tweet = hapus_katadouble(tweet)#hapus repetisi karakter
    tweet = tweet.lower()#ubah jadi lower case
    return tweet

def hapus_tanda(tweet): 
	tanda_baca = set(string.punctuation)
	tweet = ''.join(ch for ch in tweet if ch not in tanda_baca)
	return tweet

def hapus_katadouble(s): 
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

def hapus_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
						   u"\U00002500-\U00002BEF"  # chinese char
						   u"\U00002702-\U000027B0"
						   u"\U00002702-\U000027B0"
						   u"\U000024C2-\U0001F251"
						   u"\U0001f926-\U0001f937"
						   u"\U00010000-\U0010ffff"
						   u"\u2640-\u2642"
						   u"\u2600-\u2B55"
						   u"\u200d"
						   u"\u23cf"
						   u"\u23e9"
						   u"\u231a"
						   u"\ufe0f"  # dingbats
						   u"\u3030"
						   u"\x01"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


def kbbi(token): 
	#kbba = [kamus.strip('\n').strip('\r') for kamus in open('kamus\\kbba.txt')]
	kbba = [kamus.strip('\n').strip('\r') for kamus in open('C:\\Users\\navi\\Coba Coding 12 Juli 2020_menggunakan data beneran\\kbba.txt')]
	#ubah list menjadi dictionary 
	dic={}
	for i in kbba: 
		(key,val) = i.split('\t')
		dic[str(key)] = val
	#kbbi cocokan 
	final_string = ' '.join(str(dic.get(word, word)) for word in token).split()
	return final_string

def get_fitur(tweets):
	tokens = tweets.split()
	tokens = kbbi(tokens)
	return tokens

In [6]:
tweet_hslPre = []
for row in NewTrain['Tweet']:
    hapus_emoticon = hapus_emoji(row)
    pre = preprocess(hapus_emoticon)
    fitur = get_fitur(pre)
    #tweet_hslPre = ' '.join(fitur)
    tweet_hslPre.append(' '.join(fitur))
    #print(tweet_hslPre)

In [7]:
NewTrain['Tweet'] = tweet_hslPre
NewTrain

,Tweet,Label
0,ini kenapa internet rumah saya lambat banget y...,pos
1,cara berhenti berlangganan bagaimana caranya ya,pos
2,telfon nomor di atas di suruh lewat twitter le...,neg
3,hai admin sepertinya sekarang sudah ok,pos
4,assalamualaikum nomer pelanggan,pos
...,...,...
1167,bahasa ditulis penuh berhatihati tapi baca pen...,pos
1168,demmit sedih laa cerita menahan air mata maskulin,pos
1169,bahasa bintang jalan cerita bintang,pos
1170,tenggelam baitbait lemas dugaan luluh hatiku p...,net


In [8]:
#menghapus â€/titik (...) di akhir kalimat
end_stringTrainData = []
for row in NewTrain.Tweet:
    valid_characters = string.printable
    start_string = row
    end_stringTrainData.append(''.join(i for i in start_string if i in valid_characters))
    #print(end_string)
#=====End of menghapus Karakter â€¦=========
NewTrain['Tweet'] = end_stringTrainData

In [9]:
NewTrain

,Tweet,Label
0,ini kenapa internet rumah saya lambat banget y...,pos
1,cara berhenti berlangganan bagaimana caranya ya,pos
2,telfon nomor di atas di suruh lewat twitter le...,neg
3,hai admin sepertinya sekarang sudah ok,pos
4,assalamualaikum nomer pelanggan,pos
...,...,...
1167,bahasa ditulis penuh berhatihati tapi baca pen...,pos
1168,demmit sedih laa cerita menahan air mata maskulin,pos
1169,bahasa bintang jalan cerita bintang,pos
1170,tenggelam baitbait lemas dugaan luluh hatiku p...,net


In [10]:
NewTrain.to_csv("C:\\Users\\navi\\Coba Coding 12 Juli 2020_menggunakan data beneran\\CleanedModel.csv", index=False)

In [11]:
#stop = stopwords.words('indonesian')
#NewTrain['Tweet'] = NewTrain['Tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [12]:
NewTrain['Tweet'][27]

'provider internet paling jelek menurut kalian apa teman-teman kalau saya sih kenapa tidak ada uang obat '

In [13]:
myListDataTrain = []
for i in NewTrain:
    myListDataTrain = [tuple(x) for x in NewTrain.to_records(index=False)]

In [14]:
myListDataTrain

[('ini kenapa internet rumah saya lambat banget ya tolong di cek id', 'pos'),
 ('cara berhenti berlangganan bagaimana caranya ya', 'pos'),
 ('telfon nomor di atas di suruh lewat twitter lewat twitter suruh hubungi nomor di atas yang benar yang mana',
  'neg'),
 ('hai admin sepertinya sekarang sudah ok', 'pos'),
 ('assalamualaikum nomer pelanggan', 'pos'),
 ('ngelag saja terus', 'neg'),
 ('all service tv dan internet turun dari jam mohon bantuan penyelesaiannya id pelanggan',
  'pos'),
 ('mohon cek dm nya admin', 'pos'),
 ('dengan terus berlangganan atau memakai telkomsel berarti anda menyukseskan apa yang menjadi tujuan orang orang ini',
  'net'),
 ('oi mati lagi ini internetnya nomor pelanggan', 'neg'),
 ('dengan terus berlangganan atau memakai telkomsel berarti anda menyukseskan apa yang menjadi tujuan orang orang ini',
  'net'),
 ('kasian hidupku penuh dengan kebohongan lihat ini', 'neg'),
 ('sepedanya merk ya', 'net'),
 ('smartcardnya sudah dibersihin tapi tetep tidak bisa admin te

In [15]:
kl = NaiveBayesClassifier(myListDataTrain)

# Testing 

In [53]:
#TESTING DENGAN DATA SEDIKIT
Testing2 = [('halo wifi rumah jam siang sampe malem gabisa dipake nih'),
           ('seru dan asik'),
           ('hi admin wifi sudah oke sekarang'),
           ('internet sudah normal kembali'),
           ('Terima kasih'),
           ('cek dm')]

In [54]:
for komen in Testing2: 
    HslKlasifikasi = kl.classify(komen)
    print(HslKlasifikasi)

neg
net
net
pos
net
net


In [80]:
Testing3 = pd.read_csv("C:\\Users\\navi\\Coba Coding 12 Juli 2020_menggunakan data beneran\\CleanedTestingSet.csv", delimiter=',', encoding='utf-8')
Testing3

,Tweet,LabelManual
0,sudah restart dan ikutin apa kata dm dan belum...,neg
1,kalau kaya begitu knapa ya,net
2,kenapa sering banget gangguan ya kami semua ma...,neg
3,cantik banget sudah bisa akses netflix tanpa r...,pos
4,parah memang si revi ngejelek jelekin kamu ndi,net
...,...,...
95,admin saya dengan nomor pelanggan tiba tiba in...,neg
96,bahagia anniversary yang ke open all channel d...,pos
97,klau memang boat mau puts kena charge bilang d...,neg
98,om tolong di cek ini wifi kenapa mati nyala no...,net


In [56]:
#myNewlistTestSet = []
#for i in Testing3:
#    myNewlistTestSet = [tuple(x) for x in Testing3.to_records(index=False)]

In [57]:
#myNewlistTestSet

In [58]:
#for komen in myNewlistTestSet: #klasifikasi var mylistTesSet
#    HslKlasifikasi = kl.classify(komen)
#    print(HslKlasifikasi)

In [102]:
HslKlasifikasiNB = []
for komen in Testing3.Tweet: 
    HslKlasifikasi = kl.classify(komen)
    HslKlasifikasiNB.append(HslKlasifikasi)
    #print(HslKlasifikasiNB)

neg
net
neg
pos
net
neg
neg
neg
net
neg
net
pos
net
neg
neg
neg
neg
net
neg
net
net
net
neg
net
neg
net
net
neg
net
net
net
net
neg
net
neg
neg
neg
net
net
neg
neg
net
pos
neg
neg
neg
neg
neg
net
neg
net
net
neg
neg
neg
net
net
net
net
net
neg
net
neg
neg
neg
net
neg
net
net
net
neg
pos
pos
neg
pos
pos
pos
neg
neg
net
net
net
net
net
net
neg
net
net
net
neg
net
net
net
net
net
neg
pos
neg
neg
neg


In [100]:
Testing3['SentimenHslNB'] = pd.DataFrame(HslKlasifikasiNB, columns=['SentimenHslNB'])

In [101]:
Testing3

,Tweet,LabelManual,SentimenHslNB
0,sudah restart dan ikutin apa kata dm dan belum...,neg,neg
1,kalau kaya begitu knapa ya,net,net
2,kenapa sering banget gangguan ya kami semua ma...,neg,neg
3,cantik banget sudah bisa akses netflix tanpa r...,pos,pos
4,parah memang si revi ngejelek jelekin kamu ndi,net,net
...,...,...,...
95,admin saya dengan nomor pelanggan tiba tiba in...,neg,neg
96,bahagia anniversary yang ke open all channel d...,pos,pos
97,klau memang boat mau puts kena charge bilang d...,neg,neg
98,om tolong di cek ini wifi kenapa mati nyala no...,net,neg
